In [ ]:
!pip install ragatouille
!git clone 

In [ ]:
from ragatouille import RAGPretrainedModel


indexes = []
GROUP_NUM = 3 # 1, 2, or 3

for i in range(GROUP_NUM+1):
    path_to_index = ".ragatouille/colbert/indexes/GROUP{}_cluster{}".format(GROUP_NUM, i)
    RAG = RAGPretrainedModel.from_index(path_to_index)
    indexes.append(RAG.as_langchain_retriever(k=3))

In [ ]:
from langchain_core.retrievers import BaseRetriever, RetrieverLike, RetrieverOutputLike
from langchain_core.language_models import BaseLLM
from langchain_core.embeddings import Embeddings
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from typing import List



class CustomRetriever(BaseRetriever):


    vectorstore : List[RetrieverLike]

    def flatten_extend(self, matrix):
        flat_list = []
        for row in matrix:
            flat_list.extend(row)
        return flat_list

    def _get_relevant_documents(self, query: str, *, run_manager: CallbackManagerForRetrieverRun) -> List[Document]:

        all_docs = []
        for i in self.vectorstore:
            all_docs.append(i.get_relevant_documents(query, k=3))

        all_docs = self.flatten_extend(all_docs)
        return all_docs

In [ ]:
customRetriever = CustomRetriever(vectorstore=indexes)

In [ ]:
questionsFileName = "FINALQUESTIONS.txt"


f = open(questionsFileName, "r")
questions = f.readlines()
f.close()

questions = [i.strip() for i in questions]

In [ ]:
!mkdir unlimiformer/docs

for count, i in enumerate(questions):
  if (count % 100 == 0):
    print(count)
  doc = customRetriever.get_relevant_documents(i)
  context = ""
  for d in doc:
    context += d.page_content + "\n"

  prompt = "Question: {}\n\nContext: {}\n\nAnswer: ".format(i, context)
  f = open("unlimiformer/docs/q{}.txt".format(count), "w")
  f.write(prompt)
  f.close()

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
%cd unlimiformer

!python src/run_generation.py --model_type llama --model_name_or_path meta-llama/Llama-2-7b-chat-hf \
    --prefix "<s>[INST] <<SYS>>\n You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use one sentence maximum and keep the answer CONCISE. Keep the answer CONCISE. \n<</SYS>>\n\n" \
    --prompt example_inputs/rag.txt \
    --suffix " [/INST]" --test_unlimiformer --fp16 --length 314 --layer_begin 36 \
    --index_devices 0 --datastore_device 0